### downloads all laws as xml and parses them and prints to console

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
url = 'https://www.riigiteataja.ee/lyhendid.html'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
links = soup.find_all('a')

law_url = 'https://www.riigiteataja.ee/akt/'
law_links = [link for link in links if law_url in link['href']]
zipped_law_links = [[x['href'], x.text, y.text] for x, y in zip(law_links[::2], law_links[1::2])]
zipped_law_links
for link in zipped_law_links:
    url = link[0] + '.xml'
    response = requests.get(url)
    response.encoding = 'utf-8'
    law_text = response.text
    link.append(law_text)
    print(zipped_law_links.index(link), '/', len(zipped_law_links), link[1], 'from:', url)

In [ ]:
import pandas as pd

df = pd.DataFrame(zipped_law_links)
df.to_csv('law_links.csv')

### load downloaded laws from disk

In [1]:
import pandas as pd

df = pd.read_csv('law_links.csv', names=['link', 'title', 'short-title', 'xml'], header=0, index_col=0)
df

,link,title,short-title,xml
0,https://www.riigiteataja.ee/akt/113032019027,Abieluvararegistri seadus,AVRS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
1,https://www.riigiteataja.ee/akt/127052022029,Abipolitseiniku seadus,APolS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
2,https://www.riigiteataja.ee/akt/105052022005,Advokatuuriseadus,AdvS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
3,https://www.riigiteataja.ee/akt/116122022023,"Alkoholi-, tubaka-, kütuse- ja elektriaktsiisi...",ATKEAS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
4,https://www.riigiteataja.ee/akt/104012021006,Alkoholiseadus,AS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
...,...,...,...,...
366,https://www.riigiteataja.ee/akt/123122022024,Äriregistri seadus,ÄRS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
367,https://www.riigiteataja.ee/akt/123122022033,Äriseadustik,ÄS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
368,https://www.riigiteataja.ee/akt/122032022010,Ühistranspordiseadus,ÜTS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."
369,https://www.riigiteataja.ee/akt/130122021020,Ühisveevärgi ja -kanalisatsiooni seadus,ÜVVKS,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<oigus..."


In [3]:
import xmltodict
import re

def xml_preprocess(xml):
    tag = lambda tag: fr'<({tag}[^>]*)>'
    # replace <viide> with <tavatekst> to preserve order as it's used for links in text
    xml = re.sub(tag('viide'), '<tavatekst>', xml)
    xml = re.sub(tag('/viide'), '</tavatekst>', xml)

    # remove <i> and </i> tags
    xml = re.sub(tag('i'), '', xml)
    xml = re.sub(tag('/i'), '', xml)

    # replace <sup> with _ and remove </sup> tags
    xml = re.sub(tag('sup'), '_', xml)
    xml = re.sub(tag('/sup'), '', xml)

    # remove <reavahetus/>
    xml = re.sub(tag('reavahetus'), '', xml)

    # replace non breaking space
    xml = xml.replace(u'\xa0', u' ')

    return xmltodict.parse(xml)

In [5]:
# xml parsing helper functions

from collections import namedtuple

def get_list(d, key):
    if key not in d:
        return []
    if isinstance(d[key], list):
        return d[key]
    return [d[key]]

def get_or_default(d, key, default):
    if type(d) is dict:
        if key in d:
            if d[key] is not None:
                return d[key]
    return default

def get_text(v):
    return get_or_default(v, '#text', v)
    
def get_text_from_arr(value):
    if isinstance(value, list) and len(value) > 0:
        values = [get_or_default(v, 'kuvatavTekst', v) for v in value]
        values = [get_text(v) for v in values]
        values = [v for v in values if v is not None]
        if values is not None:
            return ''.join(values)
    return ''

NUMBERING = namedtuple('Numbering', ['inactive', 'num', 'idx'])
def get_numbering(dict, key, ofList):
    number = get_or_default(dict, key, None)
    active = get_or_default(number, '@kehtiv', '1')
    numberComplex = get_text(number)
    numberIndex = get_or_default(number, '@ylaIndeks', None)
    if numberComplex is dict:
        numberComplex = ofList.index(dict) + 1
    return NUMBERING(active == '0', numberComplex, numberIndex)

def print_line(depth = 0, group = '', nr = '', nridx = ' ', text = ''):
    print('\t'*depth, group, nr, nridx, text)


In [6]:
# use chatgpt3.5 to parse in-text refs

import openai
from dotenv import dotenv_values


config = dotenv_values(".env")["OPENAI_API_KEY"]
openai.organization = "org-3O7bHGD9SwjHVDuUCNCGACC3"
openai.api_key = config

def gpt35_get_refs(user):
    system = "convert law references from the input text to the following csv format: seaduse_nimi, paragrahv, lõige, punkt. only one per line. use strings for all values. use an empty string for seaduse_nimi if the law references itself. paragraphs are marked with §. only respond with csv. e.g. notariseadus;41;; ;37;;"
    MODEL = "gpt-3.5-turbo"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
        temperature=0
    )
    return response

In [7]:
# test = "Käesoleva seaduse § 24 lõigetes 6_8 ja 6_11 nimetatud intensiivse gaasitarbimisega ettevõtja esitab hiljemalt 30. päeval sama paragrahvi lõike 6_8 punktides 1–3 ja lõikes 6_11 nimetatud sündmusest arvates aktsiisideklaratsiooni ning maksab aktsiisi."
# gpt35_get_refs(test)

In [8]:
# data structures to hold parsed xml data

from __future__ import annotations
from dataclasses import dataclass, field
from collections import namedtuple


TEXTREF = namedtuple('TextReference', ['text', 'reference'])

def get_string_from_list(generator, sep=' '):
    if generator is not None:
        l = list(generator)
        if len(l) > 0:
            return sep.join(l)
    return ""

@dataclass
class Element:
    number: int = None
    index: int = None
    text: str = None
    children: list[Element] = field(default_factory=list)
    refs: object = None

    def add(self, child: Element):
        self.children.append(child)

    def numbering(self):
        pass

    def format_text(self):
        return ""

    def get_text(self, max_length: int = 0):
        texts = []
        for child in self.children:
            child_text = child.get_text(max_length)
            if child_text is not None and len(child_text) > 0:
                texts.append(child_text)
        return texts

    def fetch_references(self):
        if self.text is not None and '§' in self.text:
            self.refs = gpt35_get_refs(self.text)
            # print(self.text)
        for child in self.children:
            child.fetch_references()


class Document(Element):
    pass


class Part(Element):
    def add(self, child):
        assert type(child) not in [Document, Part], f"Invalid child element {child}"
        super().add(child)


class Chapter(Element):
    def add(self, child):
        assert type(child) not in [Document, Part, Chapter], f"Invalid child element {child}"
        super().add(child)


class Section(Element):
    def add(self, child):
        assert type(child) not in [Document, Part, Chapter, Section], f"Invalid child element {child}"
        super().add(child)


class Paragraph(Element):
    def add(self, child):
        assert type(child) not in [Document, Part, Chapter, Section, Paragraph], f"Invalid child element {child}"
        super().add(child)
    
    def numbering(self):
        if self.number is not None:
            if self.index is not None:
                return f"§{self.number}_{self.index}."
            return f"§{self.number}."
        return ""
    
    def format_text(self):
        if self.number is not None:
            if self.text is not None:
                return f"{self.numbering()} {self.text} "
            return f"{self.numbering()} "
        return ""

    def get_text(self, max_length):
        texts = []
        text = self.format_text()
        for child in self.children:
            child_text = child.get_text()
            if len(text) + len(child_text) > max_length:
                texts.append(TEXTREF(text.strip(), self))
                text = child_text
            else:
                text += f" {child_text}".strip()
        if len(text) > 0 and len(text) > len(self.format_text()):
            texts.append(TEXTREF(text.strip(), self))
        return texts


class Subparagraph(Element):
    def add(self, child):
        assert type(child) not in [Document, Part, Chapter, Section, Paragraph, Subparagraph], f"Invalid child element {child}"
        super().add(child)

    def numbering(self):
        if self.number is not None:
            if self.index is not None:
                return f"({self.number}_{self.index})"
            return f"({self.number})"
        return ""
    
    
    def get_child_texts(self):
        for child in self.children:
            yield child.get_text()

    def format_text(self):
        if self.text is not None and len(self.text.strip()) > 0:
            return f"{self.numbering()} {self.text} {get_string_from_list(self.get_child_texts())}"
        return ""
    
    def get_text(self, max_length: int = 0):
        return self.format_text()


class Point(Element):
    def add(self, child):
        assert type(child) not in [Document, Part, Chapter, Section, Paragraph, Subparagraph, Point], f"Invalid child element {child}"
        super().add(child)
    
    def numbering(self):
        if self.number is not None:
            if self.index is not None:
                return f"{self.number}_{self.index})"
            return f"{self.number})"
        return ""
    
    def format_text(self):
        if self.text is not None and len(self.text.strip()) > 0:
            return f"{self.numbering()} {self.text}"
        return ""
    
    def get_text(self, max_length: int = 0):
        return self.format_text()

In [9]:
# xml parsing functions

def parse_points(points, parent: Element):
    for point in points:
        numbering = get_numbering(point, 'alampunktNr', points)
        texts = get_or_default(point, 'sisuTekst', {})
        text = get_text_from_arr(get_list(texts, 'tavatekst'))
        # print_line(6, 'subpoint', subp_n.num, subp_n.idx, subp_text)
        if numbering.inactive:
            continue

        element = Point(number=numbering.num, index=numbering.idx, text=text)
        parent.add(element)
        get_next(point, element)
        
def parse_subparagraphs(subparagraphs, parent: Element):
    for subparagraph in subparagraphs:
        numbering = get_numbering(subparagraph, 'loigeNr', subparagraphs)
        texts = get_or_default(subparagraph, 'sisuTekst', {})
        text = get_text_from_arr(get_list(texts, 'tavatekst'))
        # print_line(5, 'subparagraph', numbering.num, numbering.idx, text)
        if numbering.inactive:
            continue

        element = Subparagraph(number=numbering.num, index=numbering.idx, text=text)
        parent.add(element)
        get_next(subparagraph, element)

def parse_paragraphs(paragraphs, parent: Document):
    for paragraph in paragraphs:
        numbering = get_numbering(paragraph, 'paragrahvNr', paragraphs)
        title = get_or_default(paragraph, 'paragrahvPealkiri', None)
        title = get_text(title)
        # print_line(4, 'paragraph', numbering.num, numbering.idx, title)
        if numbering.inactive:
            continue

        element = Paragraph(number=numbering.num, index=numbering.idx, text=title)
        parent.add(element)
        get_next(paragraph, element)
    
def parse_sections(sections, parent: Document):
    for section in sections:
        numbering = get_numbering(section, 'jaguNr', sections)
        title = get_or_default(section, 'jaguPealkiri', None)
        title = get_text(title)
        # print_line(3, 'section', sect_n.num, sect_n.idx, sect_title)
        if numbering.inactive:
            continue
        
        element = Section(number=numbering.num, index=numbering.idx, text=title)
        parent.add(element)
        get_next(section, element)
        
def parse_chapters(chapters, parent: Document):
    for chapter in chapters:
        numbering = get_numbering(chapter, 'peatykkNr', chapters)
        title = get_or_default(chapter, 'peatykkPealkiri', None)
        title = get_text(title)
        # print_line(2, 'chapter', ch_n.nr, ch_n.idx, ch_title)
        if numbering.inactive:
            continue

        element = Chapter(number=numbering.num, index=numbering.idx, text=title)
        parent.add(element)
        get_next(chapter, element)

def parse_parts(parts, parent: Document):
    for part in parts:
        numbering = get_numbering(part, 'osaNr', parts)
        title = get_or_default(part, 'osaPealkiri', None)
        title = get_text(title)
        # print_line(1, 'part', part_n.nr, part_n.idx, partTitle)
        if numbering.inactive:
            continue

        element = Part(number=numbering.num, index=numbering.idx, text=title)
        parent.add(element)
        get_next(part, element)

KEYS = ['osa', 'peatykk', 'jagu', 'paragrahv', 'loige', 'alampunkt']
PARSERS = [parse_parts, parse_chapters, parse_sections, parse_paragraphs, parse_subparagraphs, parse_points]

def get_next(xml_dict, parent: Element):
    for idx, key in enumerate(KEYS):
        if key in xml_dict:
            PARSERS[idx](get_list(xml_dict, key), parent)

In [14]:
from collections.abc import Iterable
def flatten(xs):
    # from https://stackoverflow.com/a/2158532
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (TEXTREF)):
            yield from flatten(x)
        else:
            yield x[0]

documents = {}
for idx, row in df.iterrows():
    # print('--------------------------------------')
    # print(row['title'])
    doc = Document(text=row['title'])
    documents[row['title']] = doc
    get_next(xml_preprocess(row['xml'])['oigusakt']['sisu'], doc)
    # doc_text = doc.get_text(2048)
    #print('\n'.join(list(flatten(doc_text))))
len(documents)
# there's at least one law that isn't properly parsed and some have bad xml formatting, but it should get every paragraph

371

### chatgpt parse text references test

In [11]:
documents[list(documents.keys())[0]]

Document(number=None, index=None, text='Abieluvararegistri seadus', children=[Chapter(number='1', index=None, text='ÜLDSÄTTED', children=[Paragraph(number='1', index=None, text='Abieluvararegister', children=[Subparagraph(number='1', index=None, text='Abieluvararegister on riiklik register, kuhu kantakse abieluvaralepingus toodud ning seaduses sätestatud juhtudel varalised õigused. Abieluvararegistri eesmärk on võimaldada saada kolmandatel isikutel teavet abikaasadevahelise varasuhte ning sellest tulenevate õiguste ja kohustuste kohta. Kui abieluvararegistris puuduvad andmed abikaasade kohta, siis eeldatakse, et abikaasade varalistele suhetele kohaldatakse varaühisuse varasuhet.', children=[], refs=None), Subparagraph(number='1', index='1', text='Abieluvararegistrit peetakse elektrooniliselt.', children=[], refs=None), Subparagraph(number='2', index=None, text='Abieluvararegistri pidamise ja andmetöötluse korra kehtestabvaldkonna eest vastutav ministermäärusega.', children=[], refs=Non

In [12]:
documents[list(documents.keys())[0]].fetch_references()

In [13]:
documents[list(documents.keys())[0]]

Document(number=None, index=None, text='Abieluvararegistri seadus', children=[Chapter(number='1', index=None, text='ÜLDSÄTTED', children=[Paragraph(number='1', index=None, text='Abieluvararegister', children=[Subparagraph(number='1', index=None, text='Abieluvararegister on riiklik register, kuhu kantakse abieluvaralepingus toodud ning seaduses sätestatud juhtudel varalised õigused. Abieluvararegistri eesmärk on võimaldada saada kolmandatel isikutel teavet abikaasadevahelise varasuhte ning sellest tulenevate õiguste ja kohustuste kohta. Kui abieluvararegistris puuduvad andmed abikaasade kohta, siis eeldatakse, et abikaasade varalistele suhetele kohaldatakse varaühisuse varasuhet.', children=[], refs=None), Subparagraph(number='1', index='1', text='Abieluvararegistrit peetakse elektrooniliselt.', children=[], refs=None), Subparagraph(number='2', index=None, text='Abieluvararegistri pidamise ja andmetöötluse korra kehtestabvaldkonna eest vastutav ministermäärusega.', children=[], refs=Non